<a href="https://colab.research.google.com/github/lgeger/file/blob/main/csdn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time

def get_article_links(user_profile_url):
    links = []
    page = 1
    while True:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
        }
        response = requests.get(f"{user_profile_url}/article/list/{page}", headers=headers)
        print(response.status_code)
        # response = requests.get(f"{user_profile_url}/article/list/{page}")
        soup = BeautifulSoup(response.content, 'html.parser')
        article_tags = soup.find_all('a', href=True)
        new_links = [tag['href'] for tag in article_tags if '/lfmilaoshi/article/details/' in tag['href']]
        if not new_links:
            break
        links.extend(new_links)
        page += 1
        if page>6:
            break
        time.sleep(1)  # 避免请求过于频繁，暂停1秒
    return links

def get_article_content(article_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
    }
    response = requests.get(article_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('title').get_text().strip()
    content_tag = soup.find('div', {'id': 'content_views'})
    if content_tag:
        content = content_tag.get_text().strip()
    else:
        content = ""
    return {'title': title, 'content': content}

def clean_content(content):
    content = re.sub(r'<.*?>', '', content)
    content = re.sub(r'\s+', ' ', content).strip()
    return content

user_profile_url = 'https://blog.csdn.net/lfmilaoshi'
article_links = get_article_links(user_profile_url)

articles = []
for link in article_links:
    full_link = f"https://blog.csdn.net{link}"
    article = get_article_content(link)
    article['content'] = clean_content(article['content'])
    articles.append(article)

# 保存为JSON文件
with open('csdn_articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

# 格式化为CSV文件
data = {'question': [], 'answer': []}
for article in articles:
    data['question'].append(article['title'])
    data['answer'].append(article['content'])

df = pd.DataFrame(data)
df.to_csv('csdn_articles.csv', index=False)

print("爬取并保存完成")


200
200
200
200
200
200
爬取并保存完成
